# Unstructured Data in Snowflake 101

## Demo Overview
This notebook demonstrates how to work with unstructured data in Snowflake, primarily focusing on text-based documents. It covers:
- Creating and configuring stages for file storage
- Accessing files through different URL methods
- Analyzing unstructured data with SQL AI functions


## 1. Understanding Snowflake Stages

To leverage unstructured files in Snowflake, you can store them in an **internal** or **external** stage:

- **External stages**: Pointers to cloud storage (AWS S3, Azure Storage, Google Cloud Storage)
- **Internal stages**: Native Snowflake storage with built-in encryption

### Creating an External Stage
```sql
-- Generic syntax for external stage
CREATE STAGE mydb.myschema.mystage_ext
  URL='s3://load/files/'
  STORAGE_INTEGRATION = my_storage_int
  DIRECTORY = (
    ENABLE = true
    AUTO_REFRESH = true
  )
  COMMENT = 'External (S3) Snowflake stage for permits data';
```

### Creating an Internal Stage
```sql
-- Generic syntax for internal stage
CREATE STAGE  mydb.myschema.mystage_int 
  DIRECTORY = (
    ENABLE = true
    AUTO_REFRESH = true
   )
  ENCRYPTION = ( TYPE = 'SNOWFLAKE_SSE' ) 
  COMMENT = 'Internal Snowflake stage for permits data';
```

> **💡 Pro Tip:** Always enable the directory table - it acts as a metadata catalog for files in the stage and is essential for automated processing.

> **⚠️ Note:** Auto-refresh on internal stages is currently supported for accounts on AWS. Support for accounts on Azure is expected soon.

### 📚 Documentation References
- [Create external stage](https://docs.snowflake.com/en/user-guide/data-load-s3-create-stage)
- [Configure external stage to auto-refresh](https://docs.snowflake.com/en/user-guide/data-load-dirtables-auto-s3)

## 2. Example: External S3 Stage Configuration

Let's examine a real-world example of an external S3 stage. Key features to note:

- **📍 Location**: Points to a specific partition in an S3 bucket
- **🔐 Authentication**: Uses a storage integration object for secure AWS access  
- **🔄 Auto-refresh**: Automatically detects new files for continuous processing pipelines

The following query shows the configuration of our demo stage:

In [ ]:
--reset if necessary
rm @ADVANCED_ANALYTICS.UNSTRUCTURED.PERMITS_S3/c4;
rm @ADVANCED_ANALYTICS.UNSTRUCTURED.PERMITS_S3/c5;
ALTER STAGE ADVANCED_ANALYTICS.UNSTRUCTURED.PERMITS_S3 REFRESH;

In [ ]:
USE ROLE SYSADMIN;
-- Describe the stage configuration and filter for key properties
DESC STAGE ADVANCED_ANALYTICS.UNSTRUCTURED.PERMITS_S3;

-- Show only the most important configuration details
SELECT * 
FROM TABLE(RESULT_SCAN(LAST_QUERY_ID())) 
WHERE "parent_property" IN ('STAGE_LOCATION','STAGE_INTEGRATION','DIRECTORY');

## 3. Exploring Stage Contents

Once your stage is configured, you can explore its contents using two primary methods:

### Method 1: List the Stage
- Shows file metadata including size, last modified date, and MD5 hash
- Good for getting basic file information

### Method 2: Query the Directory Table  
- Provides structured metadata that can be joined with other tables
- Better for automated processing and analytics workflows

> **⚠️ Troubleshooting**: If you don't see expected files, ensure auto-refresh is enabled or manually refresh the stage:
> ```sql
> ALTER STAGE DB_NAME.SCHEMA_NAME.STAGE_NAME REFRESH;
> ```

In [ ]:
-- Method 1: List all files in the stage with metadata
LIST @ADVANCED_ANALYTICS.UNSTRUCTURED.PERMITS_S3;

In [ ]:
-- Method 2: Query the directory table for structured metadata
SELECT 
*
FROM DIRECTORY(@ADVANCED_ANALYTICS.UNSTRUCTURED.PERMITS_S3)
ORDER BY relative_path;

## 4. File Access Methods in Snowflake

Snowflake provides three distinct URL-based methods for accessing files in cloud storage, each designed for different use cases:

### 🔐 1. Scoped UR: BUILD_SCOPED_FILE_URL()
- **Security**: Role-based access control
- **Duration**: 24 hours (temporary)
- **Authentication**: Requires role privileges on view (not stage directly)
- **Features**: Provides audit trail in query history
- **Best for**: Controlled access within same account, data sharing, Snowsight analysis

### 🔗 2. File URL: BUILD_STAGE_FILE_URL()
- **Security**: Token-based authentication
- **Duration**: Permanent URL
- **Authentication**: Requires sufficient stage privileges + authorization token
- **Features**: Works with REST API GET requests
- **Best for**: Custom applications needing persistent file access

### 🌐 3. Pre-signed URL: GET_PRESIGNED_URL()
- **Security**: Open access (no Snowflake authentication required)
- **Duration**: Configurable expiration time
- **Authentication**: None required - completely public access
- **Features**: Simple HTTPS URL
- **Best for**: BI tools, reporting applications, public file sharing

### 📊 Security Comparison
| Method | Security Level | Duration | Authentication Required |
|--------|---------------|----------|------------------------|
| Scoped URL | Highest (Role-based) | 24 hours | Role privileges |
| File URL | Medium (Token-based) | Permanent | Stage privileges + token |
| Pre-signed URL | Lowest (Open access) | Configurable | None |

In [ ]:
-- Example: Generate a pre-signed URL for a specific file
-- This URL will be valid for 3600 seconds (1 hour)
SELECT GET_PRESIGNED_URL(
    @ADVANCED_ANALYTICS.UNSTRUCTURED.PERMITS_S3, 
    'c1.pdf', 
    3600
) as presigned_url;

In [ ]:
-- Example: Generate a pre-signeds URL for all files
-- This URL will be valid for 3600 seconds (1 hour)
SELECT 
relative_path filename,
GET_PRESIGNED_URL(
    @ADVANCED_ANALYTICS.UNSTRUCTURED.PERMITS_S3, 
    relative_path, 
    3600
) as presigned_url
FROM DIRECTORY(@ADVANCED_ANALYTICS.UNSTRUCTURED.PERMITS_S3)
ORDER BY 1;

## 5. Interactive PDF Viewer with Streamlit

Now let's build an interactive Streamlit application to view and explore the PDF files stored in our Snowflake stage. This demonstrates how to create user-friendly interfaces for unstructured data analysis.

**Note:** The pypdfium2 package should be added to your notebook packages for proper PDF rendering.


In [ ]:
import streamlit as st
import pandas as pd
import pypdfium2 as pdfium
import os
from snowflake.snowpark.context import get_active_session

# Get the active Snowflake session
session = get_active_session()

# Set up the Streamlit interface
st.title("📄 PDF Document Explorer")
st.markdown("Explore PDF documents stored in your Snowflake stage")

# Create two columns for better layout
col1, col2 = st.columns([1, 2])

with col1:
    st.subheader("📂 Available Documents")
    
    # Query the stage directory to get all PDF files
    files_query = """
    SELECT 
        relative_path,
        size,
        last_modified,
        file_url
    FROM DIRECTORY(@ADVANCED_ANALYTICS.UNSTRUCTURED.PERMITS_S3)
    WHERE relative_path LIKE '%.pdf'
    ORDER BY last_modified DESC
    """
    
    # Execute query and get results
    files_df = session.sql(files_query).to_pandas()
    
    if not files_df.empty:
        # Handle uppercase column names from Snowflake
        display_columns = []
        
        for col in files_df.columns:
            if col.upper() in ['RELATIVE_PATH', 'SIZE', 'LAST_MODIFIED']:
                display_columns.append(col)
        
        # Display file information using actual column names
        if display_columns:
            st.dataframe(
                files_df[display_columns], 
                use_container_width=True,
                hide_index=True
            )
        else:
            st.dataframe(files_df, use_container_width=True, hide_index=True)
        
        # Get the correct column name for relative_path
        relative_path_col = None
        for col in files_df.columns:
            if col.upper() == 'RELATIVE_PATH':
                relative_path_col = col
                break
        
        if relative_path_col:
            # File selector
            selected_file = st.selectbox(
                "Select a PDF to view:",
                files_df[relative_path_col].tolist(),
                key="pdf_selector"
            )
            
            # Show file details
            if selected_file:
                selected_row = files_df[files_df[relative_path_col] == selected_file].iloc[0]
                st.markdown("**File Details:**")
                
                # Handle different possible column names
                size_col = next((col for col in files_df.columns if col.upper() == 'SIZE'), None)
                modified_col = next((col for col in files_df.columns if col.upper() == 'LAST_MODIFIED'), None)
                
                if size_col:
                    st.write(f"• **Size:** {selected_row[size_col]:,} bytes")
                if modified_col:
                    st.write(f"• **Last Modified:** {selected_row[modified_col]}")
        else:
            st.error("Could not find relative_path column in the data")
            
    else:
        st.warning("No PDF files found in the stage.")

with col2:
    st.subheader("📖 Document Viewer")
    
    if not files_df.empty and 'selected_file' in locals() and selected_file:
        try:
            # Download the PDF file locally for rendering
            stage_path = f"@ADVANCED_ANALYTICS.UNSTRUCTURED.PERMITS_S3/{selected_file}"
            temp_dir = "/tmp"
            
            # Download file to local temp directory
            with st.spinner(f"Loading {selected_file}..."):
                session.file.get(stage_path, temp_dir)
                local_file_path = os.path.join(temp_dir, os.path.basename(selected_file))
            
            # Render PDF as image using pypdfium2
            if os.path.exists(local_file_path):
                st.markdown("### PDF Preview")
                
                # Generate pre-signed URL for download
                presigned_query = f"""
                SELECT GET_PRESIGNED_URL(
                    @ADVANCED_ANALYTICS.UNSTRUCTURED.PERMITS_S3, 
                    '{selected_file}', 
                    3600
                ) as presigned_url
                """
                
                url_result = session.sql(presigned_query).collect()
                if url_result:
                    presigned_url = url_result[0]['PRESIGNED_URL']
                    st.markdown(f"[📥 Download PDF]({presigned_url})")
                
                # Render PDF pages as images
                pdf_document = pdfium.PdfDocument(local_file_path)
                
                # Show first page by default, with option to navigate
                total_pages = len(pdf_document)
                
                if total_pages > 1:
                    page_num = st.slider(
                        f"Select page (1 to {total_pages}):",
                        min_value=1,
                        max_value=total_pages,
                        value=1
                    ) - 1  # Convert to 0-based index
                else:
                    page_num = 0
                
                # Render the selected page
                page = pdf_document[page_num]
                pil_image = page.render(scale=2.0).to_pil()  # Higher scale for better quality
                
                st.image(
                    pil_image, 
                    caption=f"{selected_file} - Page {page_num + 1} of {total_pages}",
                    use_column_width=True
                )
                
                # Additional file operations
                st.markdown("### 🛠️ File Operations")
                
                col_a, col_b = st.columns(2)
                
                with col_a:
                    if st.button("🔄 Refresh File List"):
                        st.rerun()
                
                with col_b:
                    if st.button("🔍 View All Pages"):
                        st.markdown("**All Pages Preview:**")
                        for i in range(min(total_pages, 3)):  # Show first 3 pages max
                            page = pdf_document[i]
                            pil_image = page.render(scale=1.0).to_pil()
                            st.image(
                                pil_image, 
                                caption=f"Page {i + 1}",
                                width=300
                            )
                            if i >= 2 and total_pages > 3:
                                st.write(f"... and {total_pages - 3} more pages")
                                break
                
                # Clean up
                pdf_document.close()
                
                # Optional: Remove temp file
                try:
                    os.remove(local_file_path)
                except:
                    pass  # Ignore cleanup errors
                    
            else:
                st.error("Failed to download PDF file")
                
        except Exception as e:
            st.error(f"Error loading PDF: {str(e)}")
            st.exception(e)
    else:
        st.info("👈 Select a PDF file from the list to view it here.")

# Add some metrics at the bottom
if not files_df.empty:
    st.markdown("---")
    st.subheader("📊 Stage Statistics")
    
    metrics_col1, metrics_col2, metrics_col3 = st.columns(3)
    
    # Find the size column (handle case variations)
    size_col = next((col for col in files_df.columns if col.upper() == 'SIZE'), None)
    
    with metrics_col1:
        st.metric("Total PDF Files", len(files_df))
    
    with metrics_col2:
        if size_col and files_df[size_col].dtype in ['int64', 'float64']:
            total_size_mb = files_df[size_col].sum() / (1024 * 1024)
            st.metric("Total Size", f"{total_size_mb:.1f} MB")
        else:
            st.metric("Total Size", "N/A")
    
    with metrics_col3:
        if size_col and files_df[size_col].dtype in ['int64', 'float64']:
            avg_size_kb = files_df[size_col].mean() / 1024
            st.metric("Average File Size", f"{avg_size_kb:.1f} KB")
        else:
            st.metric("Average File Size", "N/A")


## 6. AI-Powered Document Classification with [AI_EXTRACT](https://docs.snowflake.com/en/sql-reference/functions/ai_extract)

Now let's see how we can apply one of Snowflake's powerful features for unstructured data analysis: **AI_EXTRACT function**. This function can automatically analyze documents and extract structured information using advanced AI models. It uses a zero-shot approach at classification and information extraction. For few-shot classification, you can use [AI_CLASSIFY](https://docs.snowflake.com/en/sql-reference/functions/ai_classify). For more complex extraction, such as extracting tables from images/pdfs and training models on complex documents, use [Document AI](https://docs.snowflake.com/en/user-guide/snowflake-cortex/document-ai/overview) (more on that later).

AI_EXTRACT Key capabilities:

* Extracts data from PDFs, images, Word docs, emails, and more
* Works with multiple languages
* Lets you define exactly what information to pull out
* Processes files directly without moving data around
* Handles complex extraction tasks in a single operation

Syntax examples: 
```sql
--extract info from text
SELECT AI_EXTRACT(
 text => 'John Smith lives in San Francisco and works for Snowflake',
 responseFormat => {'name': 'What is the first name of the employee?', 'city': 'What is the address of the employee?'}
);
```

```sql
--extract info from file
SELECT AI_EXTRACT(
  file => TO_FILE('@db.schema.files','document.pdf'),
  responseFormat => [['name', 'What is the first name of the employee?'], ['city', 'Where does the employee live?']]
);
```

In [ ]:
-- Example 1: Document Classification
-- Let's classify what type of document we're looking at
SELECT 
    'c1.pdf' as filename,
    AI_EXTRACT(
        file => TO_FILE('@ADVANCED_ANALYTICS.UNSTRUCTURED.PERMITS_S3', 'c1.pdf'),
        responseFormat => [[
            'document_type', 
                'What type of document is this? Choose from: 
                permit_application, 
                contractor_license, 
                invoice, 
                contract, 
                inspection_report, 
                or other'
        ]]
    ) as classification_result;

In [ ]:
-- Example 2: Extract Key Information from multiple documents
-- Now let's extract specific data fields from the document

SELECT 
    relative_path as filename,
    AI_EXTRACT(
        file => TO_FILE('@ADVANCED_ANALYTICS.UNSTRUCTURED.PERMITS_S3', relative_path),
        responseFormat => [
            ['document_title', 'What is the title or header of this document?'],
            ['applicant_name', 'Who is the applicant or main person named in this document?'],
            ['company_name', 'What is the company or business name mentioned?'],
            ['phone_number', 'What phone number is provided?'],
            ['email_address', 'What email address is listed?'],
            ['address', 'What is the main address mentioned in the document?'],
            ['date', 'What date is prominently displayed on the document?'],
            ['has_signature', 'Is there a signature present on this document? (true/false)'],
            ['document_summary', 'Provide a brief 1-2 sentence summary of what this document is about']
        ]
    ) as extracted_data
    from directory(@ADVANCED_ANALYTICS.UNSTRUCTURED.PERMITS_S3)
    limit 3;


In [ ]:
-- Example 3: Process Multiple Documents with Classification and Extraction
-- This query classifies multiple documents and extracts basic info from each

SELECT 
    relative_path filename,
    AI_EXTRACT(
        file => TO_FILE('@ADVANCED_ANALYTICS.UNSTRUCTURED.PERMITS_S3', relative_path),
        responseFormat => [
            ['document_type', 
                'What type of document is this? Choose from: 
                permit_application, 
                contractor_license, 
                invoice, 
                contract, 
                inspection_report, 
                or other'
            ],
            ['main_person', 'Who is the main person or applicant mentioned?'],
            ['organization', 'What company or organization is involved?'],
            ['key_date', 'What is the most important date mentioned?'],
            ['confidence', 'How confident are you in the document type classification? (high/medium/low)']
        ]
    ) as extracted_info_json,
    extracted_info_json:response.document_type::string document_type,
    extracted_info_json:response.key_date::date key_date,
    extracted_info_json:response.main_person::string main_person,
    extracted_info_json:response.organization::string organization,
FROM DIRECTORY(@ADVANCED_ANALYTICS.UNSTRUCTURED.PERMITS_S3)
WHERE filename LIKE '%.pdf'  
ORDER BY filename
LIMIT 3;


## 🔄 Building Incremental Processing Pipelines

### **Change Data Capture for Files**
Snowflake provides powerful change data capture capabilities on stages that enable building incremental processing pipelines. This is accomplished by creating **streams** on top of stages.

### **How It Works:**
🎯 **Step 1**: New files are added to the stage  
📊 **Step 2**: Stream automatically detects the changes  
⚡ **Step 3**: Process only the new files (not existing ones)  
🔄 **Step 4**: Stream auto-flushes after data consumption  

### **Key Benefits:**
- 🎯 **Automatic Detection**: Streams automatically track new files added to stages
- ⚡ **Incremental Processing**: Only process new files, not the entire stage
- 🔄 **Self-Managing**: Streams automatically flush after data consumption
- 📊 **Cost Effective**: Avoid reprocessing existing documents

### **Implementation Steps:**
1. Create a stream on your stage
2. Query the stream to see new files
3. Process the new files with your logic
4. Stream automatically resets for the next batch

Let's see this in action:

In [ ]:
--let's look again at the contents of our stage PERMITS_S3
SELECT * FROM DIRECTORY(@ADVANCED_ANALYTICS.UNSTRUCTURED.PERMITS_S3)

In [ ]:
--let's do an initial bulk load of document info into a table
CREATE OR REPLACE TABLE ADVANCED_ANALYTICS.UNSTRUCTURED.PERMIT_INFO 
AS
SELECT 
    relative_path filename,
    AI_EXTRACT(
        file => TO_FILE('@ADVANCED_ANALYTICS.UNSTRUCTURED.PERMITS_S3', relative_path),
        responseFormat => [
            ['document_type', 
                'What type of document is this? Choose from: 
                permit_application, 
                contractor_license, 
                invoice, 
                contract, 
                inspection_report, 
                or other'
            ],
            ['main_person', 'Who is the main person or applicant mentioned?'],
            ['organization', 'What company or organization is involved?'],
            ['key_date', 'What is the most important date mentioned?'],
            ['confidence', 'How confident are you in the document type classification? (high/medium/low)']
        ]
    ) as extracted_info_json,
    extracted_info_json:response.document_type::string document_type,
    extracted_info_json:response.key_date::date key_date,
    extracted_info_json:response.main_person::string main_person,
    extracted_info_json:response.organization::string organization,
FROM DIRECTORY(@ADVANCED_ANALYTICS.UNSTRUCTURED.PERMITS_S3)
WHERE filename LIKE '%.pdf' ;

SELECT COUNT(*) FROM ADVANCED_ANALYTICS.UNSTRUCTURED.PERMIT_INFO;

In [ ]:
--let's create a new stream on the stage
CREATE OR REPLACE STREAM ADVANCED_ANALYTICS.UNSTRUCTURED.PERMITS_S3_STREAM ON STAGE ADVANCED_ANALYTICS.UNSTRUCTURED.PERMITS_S3;

In [ ]:
--initially, the stream is empty
SELECT * FROM ADVANCED_ANALYTICS.UNSTRUCTURED.PERMITS_S3_STREAM;

In [ ]:
--lets copy a few files from another stage into the PERMITS_S3 stage
--this is simulating the files being dropped into storage
COPY FILES INTO @ADVANCED_ANALYTICS.UNSTRUCTURED.PERMITS_S3 from @ADVANCED_ANALYTICS.UNSTRUCTURED.PERMITS_SNOWFLAKE;

In [ ]:
--in a few seconds, Snowflake will receive a notification from S3 about the new files, and the stream will show the new files added to the stage
--this typically takes 30-60 seconds
--if you want to speed things up, you can run:
--ALTER STAGE ADVANCED_ANALYTICS.UNSTRUCTURED.PERMITS_S3 REFRESH;

SELECT * FROM ADVANCED_ANALYTICS.UNSTRUCTURED.PERMITS_S3_STREAM

In [ ]:
--process the files incrementally from the stream - extract the info and insert into the table
INSERT INTO ADVANCED_ANALYTICS.UNSTRUCTURED.PERMIT_INFO 
SELECT 
    relative_path filename,
    AI_EXTRACT(
        file => TO_FILE('@ADVANCED_ANALYTICS.UNSTRUCTURED.PERMITS_S3', relative_path),
        responseFormat => [
            ['document_type', 
                'What type of document is this? Choose from: 
                permit_application, 
                contractor_license, 
                invoice, 
                contract, 
                inspection_report, 
                or other'
            ],
            ['main_person', 'Who is the main person or applicant mentioned?'],
            ['organization', 'What company or organization is involved?'],
            ['key_date', 'What is the most important date mentioned?'],
            ['confidence', 'How confident are you in the document type classification? (high/medium/low)']
        ]
    ) as extracted_info_json,
    extracted_info_json:response.document_type::string document_type,
    extracted_info_json:response.key_date::date key_date,
    extracted_info_json:response.main_person::string main_person,
    extracted_info_json:response.organization::string organization,
FROM ADVANCED_ANALYTICS.UNSTRUCTURED.PERMITS_S3_STREAM;

In [ ]:
SELECT * FROM ADVANCED_ANALYTICS.UNSTRUCTURED.PERMIT_INFO;

In [ ]:
--stream is now flushed
SELECT * FROM ADVANCED_ANALYTICS.UNSTRUCTURED.PERMITS_S3_STREAM;

In [ ]:
-- 🤖 Create an automated task that runs the INSERT when stream has data
-- This creates a fully automated pipeline that processes new files as they arrive

CREATE OR REPLACE TASK ADVANCED_ANALYTICS.UNSTRUCTURED.PROCESS_NEW_PERMITS_TASK
    -- Task configuration
    -- WAREHOUSE = 'COMPUTE_WH'  -- warehouses are now optional in most cases, use serverless!
    SCHEDULE = '5 MINUTE'     -- Check every 5 minutes
    -- Condition: Only run when stream has data
    WHEN SYSTEM$STREAM_HAS_DATA('ADVANCED_ANALYTICS.UNSTRUCTURED.PERMITS_S3_STREAM')
AS
    -- The INSERT statement from our previous example
    INSERT INTO ADVANCED_ANALYTICS.UNSTRUCTURED.PERMIT_INFO 
    SELECT 
        relative_path filename,
        AI_EXTRACT(
            file => TO_FILE('@ADVANCED_ANALYTICS.UNSTRUCTURED.PERMITS_S3', relative_path),
            responseFormat => [
                ['document_type', 
                    'What type of document is this? Choose from: 
                    permit_application, 
                    contractor_license, 
                    invoice, 
                    contract, 
                    inspection_report, 
                    or other'
                ],
                ['main_person', 'Who is the main person or applicant mentioned?'],
                ['organization', 'What company or organization is involved?'],
                ['key_date', 'What is the most important date mentioned?'],
                ['confidence', 'How confident are you in the document type classification? (high/medium/low)']
            ]
        ) as extracted_info_json,
        extracted_info_json:response.document_type::string document_type,
        extracted_info_json:response.key_date::date key_date,
        extracted_info_json:response.main_person::string main_person,
        extracted_info_json:response.organization::string organization
    FROM ADVANCED_ANALYTICS.UNSTRUCTURED.PERMITS_S3_STREAM;

-- Enable the task to start running
ALTER TASK ADVANCED_ANALYTICS.UNSTRUCTURED.PROCESS_NEW_PERMITS_TASK RESUME;

--Suspend when no longer needed
ALTER TASK ADVANCED_ANALYTICS.UNSTRUCTURED.PROCESS_NEW_PERMITS_TASK SUSPEND;

## 📋 Task Management & Monitoring

### **Key Components Explained:**

⏰ **SCHEDULE**: How often the task checks for new data (1 MINUTE, 5 MINUTE, 1 HOUR, etc.)  
🎯 **WHEN Condition**: `SYSTEM$STREAM_HAS_DATA()` ensures task only runs when there's new data  
⚡ **AUTO-FLUSH**: Stream automatically resets after successful task completion  

### **Task Management Commands:**


In [ ]:
-- 🔍 Monitor task status and execution history
SHOW TASKS LIKE 'PROCESS_NEW_PERMITS_TASK';

-- 📊 View task execution history
SELECT *
FROM TABLE(INFORMATION_SCHEMA.TASK_HISTORY(
    TASK_NAME => 'ADVANCED_ANALYTICS.UNSTRUCTURED.PROCESS_NEW_PERMITS_TASK'
))
ORDER BY SCHEDULED_TIME DESC
LIMIT 10;

-- ⏸️ Suspend the task (stop automatic execution)
-- ALTER TASK ADVANCED_ANALYTICS.UNSTRUCTURED.PROCESS_NEW_PERMITS_TASK SUSPEND;

-- ▶️ Resume the task (start automatic execution)
-- ALTER TASK ADVANCED_ANALYTICS.UNSTRUCTURED.PROCESS_NEW_PERMITS_TASK RESUME;

-- 🗑️ Drop the task completely
-- DROP TASK ADVANCED_ANALYTICS.UNSTRUCTURED.PROCESS_NEW_PERMITS_TASK;


### 🎯 **Complete Automation Achieved!**

With this task in place, you now have a **fully automated document processing pipeline**:

1. **📁 Files arrive** → S3 stage receives new documents
2. **🔄 Auto-detection** → Stream captures new file events  
3. **⚡ Task triggers** → Scheduled task checks stream every 5 minutes
4. **🤖 AI processing** → AI_EXTRACT analyzes and extracts data from new files
5. **💾 Data storage** → Results automatically inserted into PERMIT_INFO table
6. **🔄 Stream resets** → Stream flushes, ready for next batch

### 🚀 **Production Benefits:**
- **Zero Manual Intervention**: Completely hands-off processing
- **Cost Efficient**: Only runs when there's actual work to do  
- **Scalable**: Handles hundreds of documents automatically
- **Reliable**: Built-in error handling and retry logic
- **Auditable**: Full execution history and monitoring capabilities

**This is enterprise-grade document automation in action!** 🎉


### 💰 **Understanding AI_EXTRACT Costs**

#### **How AI_EXTRACT Billing Works:**
Based on the [Snowflake Cortex AISQL documentation](https://docs.snowflake.com/en/user-guide/snowflake-cortex/aisql#cost-considerations), AI_EXTRACT uses Snowflake's **credit-based billing model** for AI services:

🔢 **Token-Based Pricing**: AI_EXTRACT charges based on the number of tokens processed:
- **Input tokens**: Text extracted from your documents (files, images, PDFs)
- **Output tokens**: The structured JSON response generated by the AI model
- **Both input AND output tokens count toward your total cost**

💳 **Credit Consumption**: 
- Costs are deducted from your Snowflake credit balance
- Different models have different token-to-credit ratios (see [Consumption Table](https://www.snowflake.com/legal-files/CreditConsumptionTable.pdf))
- Charges appear in metering history under "AI Services", for example:

#### **Cost Factors That Impact Your Bill:**

📄 **Document Characteristics:**
- **File size**: Larger PDFs = more content = more input tokens
- **Text density**: Dense documents cost more than sparse ones
- **Image quality**: High-resolution images require more processing
- **Document complexity**: Multi-page, multi-column layouts increase token count

🎯 **Extraction Scope:**
- **Number of fields**: More extraction fields = higher output token count
- **Field complexity**: Detailed descriptions require more processing
- **Response format**: JSON structure adds to output token count

#### **Cost Optimization Strategies:**

✅ **Smart Batching**:
```sql
-- Process multiple documents in single query to amortize overhead
SELECT filename, AI_EXTRACT(...) FROM directory_table LIMIT 100;
```

✅ **Selective Processing**:
```sql
-- Only process files that haven't been analyzed yet
WHERE filename NOT IN (SELECT filename FROM processed_docs);
```

✅ **Efficient Field Design**:
- Use concise field descriptions
- Extract only necessary information
- Avoid redundant extraction fields

#### **Track Your AI_EXTRACT Costs:**
```sql
-- Monitor credit consumption for AI services
SELECT 
    DATE(start_time) as usage_date,
    SUM(credits_used) as total_credits,
    COUNT(*) as query_count
FROM SNOWFLAKE.ACCOUNT_USAGE.METERING_HISTORY 
WHERE service_type = 'AI_SERVICES'
  AND start_time >= DATEADD('day', -7, CURRENT_DATE())
GROUP BY DATE(start_time)
ORDER BY usage_date DESC;
```

### 🧠 **AI_EXTRACT Key Features Demonstrated:**

1. **🎯 Document Classification**: Automatically categorize documents by type
2. **📊 Structured Data Extraction**: Transform unstructured text into queryable fields
3. **🔄 Batch Processing**: Process multiple documents efficiently
4. **🎨 Flexible Schemas**: Define custom extraction fields for any document type
5. **⚡ Real-time Processing**: No need for external AI services - everything runs in Snowflake

### 💡 **Pro Tips:**
- **Specific Instructions**: The more specific your field descriptions, the better the extraction quality
- **Validation Fields**: Include confidence scores to assess extraction quality
- **Incremental Processing**: Use streams and tasks for continuous document processing
- **Error Handling**: AI_EXTRACT gracefully handles various document formats and quality levels

### 🚀 **Next Steps:**
This demonstrates the foundation for building automated document processing pipelines. You can extend this to:
- Create tables to store extracted data permanently
- Set up automated workflows with tasks and streams
- Build approval workflows for human validation
- Create dashboards for document analytics


## 7. Performance & Cost Considerations

### 💰 **Cost Optimization**
When working with unstructured data in Snowflake, consider these cost optimization strategies:

- **File Size Management**: Larger files take more compute to process with AI_EXTRACT
- **Batch Processing**: Process multiple documents in a single query when possible
- **Stage Organization**: Use partitioned stage layouts for efficient file access
- **Compression**: Store files in compressed formats when possible

### 📊 **Quality & Performance Tips**

#### **Optimal PDF Characteristics:**
- **Resolution**: 150-300 DPI for text extraction
- **Format**: Text-based PDFs perform better than scanned images
- **Size**: Files under 10MB typically process faster
- **Language**: Clear, standard fonts improve extraction accuracy

#### **Quality Monitoring:**
```sql
-- Monitor extraction confidence levels
SELECT 
    document_type,
    AVG(CASE WHEN confidence = 'high' THEN 3 
             WHEN confidence = 'medium' THEN 2 
             ELSE 1 END) as avg_confidence_score,
    COUNT(*) as document_count
FROM PERMIT_INFO
GROUP BY document_type
ORDER BY avg_confidence_score DESC;
```

### 🎯 **Best Practices Summary**
1. **Test extraction prompts** on sample documents first
2. **Monitor processing costs** using query profiling
3. **Validate results** with confidence scoring
4. **Use incremental processing** for ongoing document workflows
5. **Optimize file formats** for your specific use case


## 🎉 Conclusion

### **What You've Learned**
This notebook demonstrated the complete workflow for handling unstructured data in Snowflake:

✅ **Stage Configuration**: Created and configured external S3 stages with auto-refresh capabilities  
✅ **File Access Methods**: Explored scoped URLs, file URLs, and pre-signed URLs for different use cases  
✅ **Interactive Visualization**: Built a Streamlit app for document exploration and preview  
✅ **AI-Powered Processing**: Used AI_EXTRACT for intelligent document classification and data extraction  
✅ **Incremental Pipelines**: Implemented stream-based processing for continuous document workflows  
✅ **Performance Optimization**: Learned best practices for cost-effective unstructured data processing  

### **🚀 Ready for Production?**
You now have the foundation to build enterprise-scale unstructured data solutions:

**For Document Management Systems:**
- Automated classification and metadata extraction
- Searchable document repositories
- Compliance and audit workflows

**For Business Process Automation:**
- Invoice and contract processing
- Permit and application workflows  
- Customer document onboarding

**For Analytics & Insights:**
- Document sentiment analysis
- Content trend identification
- Regulatory compliance monitoring

### **📚 Additional Resources**
- [Snowflake Unstructured Data Guide](https://docs.snowflake.com/en/user-guide/unstructured-intro)
- [AI_EXTRACT Documentation](https://docs.snowflake.com/en/sql-reference/functions/ai_extract)
- [Document AI for Complex Extraction](https://docs.snowflake.com/en/user-guide/snowflake-cortex/document-ai/overview)
- [Snowflake Cortex AI Functions](https://docs.snowflake.com/en/user-guide/snowflake-cortex/llm-functions)

---
*Happy analyzing! 🎯*


## 🎁 Bonus: Additional Unstructured Data Capabilities

### 📄 **AI_PARSE_DOCUMENT: Parse All Text**

**Purpose**: Extract text and layout information from complex documents including PDFs, images, and scanned documents using OCR technology.

**Key Features**:
- **OCR Mode**: Extract raw text from scanned documents and images
- **LAYOUT Mode**: Preserve document structure, tables, and formatting
- **Multi-format Support**: Works with PDFs, images (PNG, JPG), and more
- **Batch Processing**: Process multiple documents efficiently

**Example Use Cases**:
```sql
-- Extract all text from a scanned document
SELECT AI_PARSE_DOCUMENT(
    @my_stage, 
    'scanned_contract.pdf', 
    {'mode': 'OCR'}
) as extracted_text;

-- Preserve document layout and structure
SELECT AI_PARSE_DOCUMENT(
    @my_stage, 
    'financial_report.pdf', 
    {'mode': 'LAYOUT'}
) as structured_content;
```

### 🎤 **AI_TRANSCRIBE: Speech-to-Text Processing**

**Purpose**: Convert audio files into text with speaker identification and timestamps.

**Key Features**:
- **Multi-format Audio**: Supports MP3, WAV, M4A, and other common formats
- **Speaker Diarization**: Identifies different speakers in conversations
- **Timestamps**: Provides precise timing for each spoken segment
- **Language Detection**: Automatically detects spoken language

**Example Use Cases**:
```sql
-- Transcribe customer service calls
SELECT AI_TRANSCRIBE(@call_recordings_stage, 'customer_call.mp3') as transcript;

-- Process meeting recordings with speaker identification
SELECT 
    filename,
    AI_TRANSCRIBE(@meeting_stage, filename) as meeting_transcript
FROM DIRECTORY(@meeting_stage) 
WHERE filename LIKE '%.mp3';
```

**Business Applications**:
- Customer service call analysis
- Meeting transcription and summarization
- Compliance and audit documentation
- Voice-to-text data entry automation

### 🖼️ **AI_COMPLETE Multimodal: Advanced Image Analysis**

**Purpose**: Analyze images using large language models for object detection, image comparison, and visual content understanding.

**Key Capabilities**:
- **Object Detection**: Identify and describe objects, people, text in images
- **Image Comparison**: Compare multiple images for similarities/differences
- **Scene Analysis**: Understand context and activities in visual content
- **OCR Integration**: Extract and interpret text within images

**Example Use Cases**:
```sql
-- Analyze product images for inventory management
SELECT AI_COMPLETE(
    'llama3.1-70b',
    TO_FILE(@product_images, 'product_001.jpg'),
    'Describe this product in detail. What is it? What condition is it in? List any visible defects.'
) as product_analysis;

-- Compare before/after images for damage assessment
SELECT AI_COMPLETE(
    'claude-3-haiku',
    [TO_FILE(@damage_photos, 'before.jpg'), TO_FILE(@damage_photos, 'after.jpg')],
    'Compare these two images. What changes or damage do you observe between the before and after photos?'
) as damage_assessment;
```

**Business Applications**:
- Insurance claim processing with photo analysis
- Quality control in manufacturing
- Medical imaging analysis and reporting
- Real estate property assessment

### 🔍 **AI_SIMILARITY: Advanced Similarity Search**

**Purpose**: Compute similarity scores between texts, images, or mixed content using vector embeddings.

**Key Features**:
- **Multi-modal Support**: Compare text-to-text, image-to-image, or text-to-image
- **Vector Embeddings**: Uses advanced embedding models for accurate similarity
- **Batch Comparisons**: Efficiently compare multiple items at once
- **Cosine Similarity**: Industry-standard similarity scoring

**Example Use Cases**:
```sql
-- Find similar documents in your corpus
SELECT 
    doc1.filename,
    doc2.filename,
    AI_SIMILARITY(doc1.content, doc2.content) as similarity_score
FROM documents doc1 
CROSS JOIN documents doc2
WHERE doc1.filename != doc2.filename
  AND AI_SIMILARITY(doc1.content, doc2.content) > 0.8;

-- Image similarity for duplicate detection
SELECT 
    image_name,
    AI_SIMILARITY(
        TO_FILE(@image_stage, image_name),
        TO_FILE(@reference_stage, 'reference_image.jpg')
    ) as similarity_score
FROM DIRECTORY(@image_stage)
WHERE similarity_score > 0.9;
```

**Business Applications**:
- Duplicate content detection
- Recommendation systems  
- Fraud detection through pattern matching
- Content organization and categorization

### 🚀 **Complete Workflow Example**

Here's how you might combine these capabilities in a comprehensive document processing pipeline:

```sql
-- Comprehensive document processing workflow
WITH document_processing AS (
    -- Step 1: Parse document structure
    SELECT 
        filename,
        AI_PARSE_DOCUMENT(@docs_stage, filename, {'mode': 'LAYOUT'}) as parsed_content
    FROM DIRECTORY(@docs_stage)
    WHERE filename LIKE '%.pdf'
),
extracted_data AS (
    -- Step 2: Extract structured information
    SELECT 
        filename,
        parsed_content,
        AI_EXTRACT(
            parsed_content,
            [
                ['document_type', 'What type of document is this?'],
                ['key_entities', 'List the main entities (people, companies, dates)'],
                ['summary', 'Provide a brief summary']
            ]
        ) as extracted_info
    FROM document_processing
),
similarity_analysis AS (
    -- Step 3: Find similar documents
    SELECT 
        d1.filename,
        d1.extracted_info,
        ARRAY_AGG(d2.filename) as similar_documents
    FROM extracted_data d1
    JOIN extracted_data d2 
        ON AI_SIMILARITY(d1.parsed_content, d2.parsed_content) > 0.8
        AND d1.filename != d2.filename
    GROUP BY d1.filename, d1.extracted_info
)
SELECT * FROM similarity_analysis;
```

### 📚 **Additional Resources**

- [AI_PARSE_DOCUMENT Documentation](https://docs.snowflake.com/en/sql-reference/functions/ai_parse_document)
- [AI_TRANSCRIBE Documentation](https://docs.snowflake.com/en/sql-reference/functions/ai_transcribe)  
- [AI_COMPLETE Multimodal Guide](https://docs.snowflake.com/en/user-guide/snowflake-cortex/aisql)
- [AI_SIMILARITY Documentation](https://docs.snowflake.com/en/sql-reference/functions/ai_similarity)

---
**🎯 These advanced capabilities transform Snowflake into a complete unstructured data platform, enabling sophisticated AI-powered analysis across text, audio, images, and documents!**
